In [ ]:
!pip install --upgrade google-cloud-aiplatform[evaluation,agent_engines,adk]

In [ ]:
from vertexai import rag
from vertexai.generative_models import GenerativeModel, Tool
from vertexai import Client, types
import vertexai
from vertexai import agent_engines

In [ ]:
PROJECT_ID="pjt-poc-lgcaip-gempoc" # @param {type:"string"}
LOCATION="us-central1" # @param {type:"string"}
RESOURCE_ID = "projects/322934811602/locations/us-central1/reasoningEngines/1564531379046187008" 

In [ ]:
# Initialize Vertex AI API once per session
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
import pandas as pd

eval_dataset = pd.DataFrame({
    "prompt": [
        "LG화학 현장 전문가 육성 제도 알려주세요.",
        "급여 지급 기준이 어떻게 되나요",
        "노트북 사용중 모니터 추가 요청하는 방법",
        "우리회사에서 책임에게 요구되는 역할에 대해서 설명해줘.",
        "가족 돌봄 휴가라는 것이 있던데 어떤 상황에서 사용하면 좋은지 설명해줘."
    ],
    "reference": [
        """
        LG화학 현장 전문가 육성제도에 대한 안내입니다.
        
        현장 전문가 육성제도는 탁월한 전문성을 보유한 구성원이 사내에서 전문가로서 인정받고 지속적으로 성장할 수 있도록 돕는 제도입니다.
        LG화학 명장은 생산/품질/공무·설비 분야에 종사하는 전문/생산기술직 중 
        1) 탁월한 전문성으로 사업 성과 창출 공적이 큰 자
        2) 존경받는 인품으로 타의 모범이 되는 자
        중 선발합니다.
        
        LG화학 전문가는 생산/품질/공무·설비 분야에 종사하는 전문/생산기술직 중
        1) 개선 PJT 리딩이 가능한 전문성을 보유한 자
        2) MGB 자격 보유자 
        중 선발합니다.
        """,
        """
        급여/상여 지급에 대한 안내입니다.
        
        ▶ 급여 기산일
        : 당월 1일 ~ 말일 근무일수에 대해 25일 지급
        
        ▶ 근태 지급 기준
        : 전월 21일 ~ 당월 20일 근태수당에 대해 25일 지급
        
        단,사무직의 경우에는 1,3,5,7,9,11월에 OT로 발생한 보상휴가를 2개월간 사용 후 미사용분에 대해서 보상휴가비 지급
        """,
        """
        모니터 신청에 대한 안내입니다.

        ▶ 전용 리스 모니터 신청 : 임직원 1명당 1대 지급 (신규)
        * 신청경로 : G-Portal→ MY Asset→ Hardware 관련 신청→ PC/모니터 신청→ 전용 리스 PC/ 모니터 신청
        <a href="http://pcasset.lgchem.com/index.do;jsessionid=EC0B8B06E77CDD41640B1A03D30A353D.4b078a14e28167161" target="_blank">[My Asset 바로가기 >>>]</a>
        """,
        """
        LG화학의 '책임'은 특정 직무 영역에서 전문 지식과 경험을 보유한 전문가 역할을 수행합니다. L3 직급에 해당하며, 문제 발생 시 Best Practice와 직무 지식을 활용하여 해결하고, 기존 방식을 넘어서는 새로운 관점으로 복잡한 문제까지 해결할 수 있어야 합니다. 또한, 독립적으로 업무를 수행하며 최소한의 업무 지침만을 받습니다.
        
        '책임'에게 요구되는 역할은 다음과 같이 요약할 수 있습니다.
        
        전문성: 특정 직무 영역에 대한 깊이 있는 전문 지식과 실무 경험을 보유하고, 이를 바탕으로 개선 방향을 제시합니다.
        
        문제 해결 능력: 다양한 정보를 종합적으로 분석하고 새로운 관점을 적용하여 복잡한 문제를 해결합니다.
        
        리더십과 독립성: 도전적으로 과제를 기획하고 추진하며 팀 내 협업을 이끌 수 있습니다. 대부분의 업무를 독립적으로 수행하며, 세부적인 가이드라인 없이도 업무 수행이 가능합니다.
        
        영향력 및 소통: 소속 조직 및 유관 부서에 직접적인 영향을 미치며, 복잡한 정보를 명확하게 설명하고 의견을 조율하여 합의를 이끌어내는 역량이 필요합니다.
        """,
        """
        가족 돌봄 휴가는 임직원 본인의 조부모, 부모, 배우자, 배우자의 부모, 자녀 또는 손자녀가 질병, 사고, 노령으로 인해 긴급하게 돌봄이 필요한 경우에 사용할 수 있는 휴가입니다.
        
        연간 최대 10일까지 1일 단위로 분할하여 사용할 수 있으며, 이 휴가는 무급으로 운영됩니다. 사용한 휴가 일수는 가족 돌봄 휴직 기간에 포함됩니다.
        """
    ],
    "response": [
        "",
        "",
        "",
        "",
        "",
    ]
})

In [ ]:
try:
    # Use the 'open' method to get a client for a deployed agent
    my_agent_client = agent_engines.AgentEngine.open(
        resource_name=RESOURCE_ID
    )
except Exception as e:
    print(f"Error accessing agent: {e}")

In [ ]:
for i in range(len(eval_dataset)):
  query_response = my_agent_client.query(eval_dataset["prompt"][i])
  eval_dataset.loc[i, "response"] = response.text